# calculate History term
In this  notebook the history force is calculated given by: 
$$ \mathbf{F}_{history}  = 3\pi \nu_f d \int_{0}^t K(t-s)\frac{d}{ds} (\mathbf{u}_p(s) -\mathbf{u}_s(s)) ds $$ 
with the "Kernel" K depends on the particle Reynolds number, for small particle Reynolds number we have the Basset version:
$$ K_{Basset} = \frac{d}{2 \pi \sqrt{t-s}}$$
For larger particle Reynolds number different the history kernel of Mei1992 is used 
$$ \mathbf{F}_{history}  = 3\pi \nu_f d \int_{0}^t K(t-s)\frac{d}{ds} (\mathbf{u}_p(s) -\mathbf{u}_s(s)) ds $$ 
where there are slightly differnt values for coefficients c1 and c2 from literature:
- Mei 1992 : c1 = 2, c2 = 0.1015
- Kim 1996 : c1 = 2.5 c2 = 0.126
- Dorgan 2007: c1 = 2.5, c2 = 0.2
For larger Reynolds numbers there is a crossover between basset like decay (as ^0.5) at short time scales and squared  
decay (^2) on long timescales. The timescale associated with this is $\tau_h$ and makes that the history decays faster  
to zero for larger particles reynolds numbers, this allows to use a finite window to calculate the history term from  
time $t-\tau_h$ to current time $t$.


idea for analysis which you might want to do later if you get questions about how valid it is to ignore it?: does history term work in other direaction than slip velocity? Angle analysis

In [ ]:
# import needed packages

#update reading in packages when rerunning this cell
%load_ext autoreload
%autoreload 2

import numpy as np
import xarray as xr 
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.cm as cm
from matplotlib import colormaps
import cartopy.crs as ccrs #for plotting on map
import cartopy as cart
import scipy.special as sc
from datetime import datetime, timedelta
from scipy.interpolate import CubicSpline # maybe not needed
import sys
sys.path.append("/nethome/4291387/Maxey_Riley_advection/Maxey_Riley_advection/simulations")
sys.path.append("/nethome/4291387/Maxey_Riley_advection/Maxey_Riley_advection/src")
from particle_characteristics_functions import stokes_relaxation_time, Re_particle, factor_drag_white1991, diffusion_time, slip_force
from history_term_functions import Basset_kernel, Mei1992_kernel, history_timescale, Daitche, Hinsberg, f_Mei1992, History_Force_Hinsberg_Mei_kernel, History_Force_Hinsberg_Basset_kernel
from analysis_functions_xr import derivative_backward, calc_tidal_av
from analysis_functions import make_PDF, make_lognormal_PDF
plt.style.use('../python_style_Meike.mplstyle')

Magma = colormaps['magma']
Magmalist = Magma(np.linspace(0.2, 0.9, 5))

In [ ]:
# analytical test functions

def dUslip_dt_cos(t):
    return np.cos(t,dtype = np.float128)
    
def Basset_analytical_cos(t):
    
    S, C = sc.fresnel(np.sqrt(2 * t / np.pi))
    return np.sqrt(2 * np.pi ) * (np.cos(t) * C + np.sin(t) * S )

    
def dUslip_dt_tt(t):
    return t*t
    
def Basset_analytical_tt(t):
    return 2/15 * np.sqrt(t) * 8 * t *t 


In [ ]:
# constants
Rearth = 6371e3 # in m,
deg2rad = np.pi / 180.
sec_in_min = 60
min_in_hour = 60
sec_in_hour = sec_in_min * min_in_hour
omega_earth =  7.2921e-5 #[rad/sec]


rho_water = 1027 # kg/m3 https://www.engineeringtoolbox.com/sea-water-properties-d_840.html (at 10 deg)
dynamic_viscosity_water = 1.41 * 10**(-3) # kg/(ms) https://www.engineeringtoolbox.com/sea-water-properties-d_840.html (at 10 deg)
kinematic_viscosity_water = dynamic_viscosity_water / rho_water
diameter =0.25 # m
B=0.68
tau_p = stokes_relaxation_time(diameter, kinematic_viscosity_water, B)
S = 1/12 * diameter* diameter/kinematic_viscosity_water

cs = {'Mei':{'c1':2,'c2':0.105},
             'Kim':{'c1':2.5,'c2':0.126},
             'Dorgan':{'c1':2.5,'c2':0.2}}

## Compare numerical solution Basset history term for idealized flow
Comparing with Fig 4 in Moreno-Casas, Bombardelli 2016

In [ ]:
Daitche_analytical_tt=[]
Hinsberg_analytical_tt=[]
Daitche_analytical_cos=[]
Hinsberg_analytical_cos=[]


T = 50 * np.pi
Nlist =  np.array([243,729,2187,6561,19683,59049,177411,1594323])
# Nlist =  np.arange(100,100000,100)
for N in Nlist:
    dt = np.float128(T/(N))
    tlist = np.arange(0,T+0.1*dt,dt)
    dudt_tt =dUslip_dt_tt(tlist)
    Daitche_analytical_tt.append(Daitche(dudt_tt,N,dt))
    Hinsberg_analytical_tt.append(Hinsberg(dudt_tt,N,dt))

    dudt_cos =dUslip_dt_cos(tlist)
    Daitche_analytical_cos.append(Daitche(dudt_cos,N,dt))
    Hinsberg_analytical_cos.append(Hinsberg(dudt_cos,N,dt))
analytical_tt =Basset_analytical_tt(T)
analytical_cos =Basset_analytical_cos(T)


 




In [ ]:
fig, ax = plt.subplots()
ax.plot(T/(Nlist), np.abs(Hinsberg_analytical_cos-analytical_cos)/analytical_cos,'--',color='orange')
ax.plot(T/(Nlist), np.abs(Daitche_analytical_cos-analytical_cos)/analytical_cos,'o',color='green')


# ax.plot(T/(Nlist), np.abs(Hinsberg_analytical_tt-analytical_tt)/analytical_tt,'--',color='navy')
# ax.plot(T/(Nlist), np.abs(Daitche_analytical_tt-analytical_tt)/analytical_tt,'o',color='firebrick')

ax.legend(['Hinsberg $\\cos(t)$','Daitche $\\cos(t)$','Hinsberg $t^2$','Daitche $t^2$',])
ax.set_xscale('log')
ax.set_yscale('log')
ax.set_xlabel('dt')

ax.set_ylabel('relative error')
# ax.set_ylim(10**(-14),10**0)
# ax.set_xlim(10**(-4),2*10**(-1))


In [ ]:
fig, ax = plt.subplots()
ax.plot((Nlist), Hinsberg_analytical_cos,'--o')
ax.set_xscale('log')


### Comparing Basset history term and Mei history term for idealized flows

In [ ]:
N = 500
T=50 * np.pi
dt = T/N
c1 = cs['Mei']['c1']
c2 = cs['Mei']['c2']

Rep = 400
S = 1 *  diameter* diameter/kinematic_viscosity_water


tlist = np.arange(0,T+0.1*dt,dt)
dudt_cos =dUslip_dt_cos(tlist)

Nwindow = 50


fmei = f_Mei1992(dudt_cos, T, c1, c2, Rep ,N ,dt ,S)
fmei_window = np.array(fmei)[-(Nwindow + 1):]
fbasset_window = np.array(dudt_cos)[-(Nwindow + 1):]
# fmei_window = [fmei[k] if k > k0 else 0 for k in range(0,N+1,1)]


daitche_Mei = Daitche(fmei,N,dt)
daitche_basset = Daitche(dudt_cos,N,dt)
Hinsberg_Mei = Hinsberg(fmei,N,dt)
Hinsberg_Mei_window = Hinsberg(fmei_window,Nwindow,dt)
Hinsberg_Basset_window = Hinsberg(fbasset_window,Nwindow,dt)

Hinsberg_basset = Hinsberg(dudt_cos,N,dt)
# print(daitche_basset)
# print(daitche_Mei)

print(f'Basset(N={N}) = {Hinsberg_basset} \t Basset(Nwindow ={Nwindow}) = {Hinsberg_Basset_window}')
print(f'Mei (N={N}) = {Hinsberg_Mei} \t Mei(Nwindow ={Nwindow}) = {Hinsberg_Mei_window}')



## checking implementation of history force unitless kernel
Below I compare the implementation of the unitless history kernel (from Dorgan 2007) against the standard version used by Hinsberg/Daitche (checked above) 


In [ ]:
# check force (in Newton!)
T = 50 * np.pi
N = 54564
dt = T/N
Rep = 400
c1 = cs['Mei']['c1']
c2 = cs['Mei']['c2']
S =  diameter* diameter/kinematic_viscosity_water
tlist = np.arange(0,T+0.1*dt,dt)
dudt_cos =dUslip_dt_cos(tlist)

# Nwindow = 50

fmei = f_Mei1992(dudt_cos, T, c1, c2, Rep ,N ,dt ,S)

Hinsberg_unit1 = np.sqrt(S/(4*np.pi)) * Hinsberg(fmei,N,dt)
Hinsberg_unit2 =  History_Force_Hinsberg_Mei_kernel(dudt_cos,T,c1,c2,Rep,N,dt,diameter, kinematic_viscosity_water)

print(Hinsberg_unit1)
print(Hinsberg_unit2)


## compare functional form Mei history kernel and and Talaei history kernel
Talaei tail does die less fast ...

In [ ]:
def Mei_history_kernel_function(s, t,diameter, dynamic_viscosity, Rep, c1, c2):
    t_d = diameter**2  / dynamic_viscosity
    fh = (0.75 + c2*Rep)**3
    return  ((4 * np.pi * (t-s)/t_d)**(1/(2*c1)) + (np.pi * (t-s)**2 * Rep**3 / (fh * t_d**2))**(1/c1))**(-c1)


def Basset_history_kernel_function(s,t,diameter,dynamic_viscosity):
    t_d = diameter**2 / dynamic_viscosity
    return (4 * np.pi * (t-s)/ t_d)**(-1/2)

def Talaei_history_kernel_function(s, t, diameter, dynamic_viscosity, Rep):
    cd = factor_drag_white1991(Rep)
    t_d = diameter**2  / dynamic_viscosity
    return cd / ((4 * np. pi *(t-s) / t_d )**(1/2) + (4 * np.pi * (Rep**2) * ((t-s)**2) / (t_d**2))**(1/2))

def Talaei_history_kernel_function_original(s, t, diameter, dynamic_viscosity, Rep):
    cd = factor_drag_white1991(Rep)
    uslip = Rep * dynamic_viscosity / diameter
    a = diameter / 2
    Ap = np.pi * a * a 
    norm = np.pi * 3 * dynamic_viscosity * diameter
    prefactor = 1/2 * Ap * a / norm 
    numerator = cd * 24/Rep * uslip
    denominator = np.sqrt(np.pi * dynamic_viscosity * (t-s))+ np.sqrt(np.pi) * uslip * (t-s)
    return prefactor * numerator / denominator

In [ ]:
fig, ax = plt.subplots()
slist= np.arange(9.9,10,0.0001)
td = diameter**2/dynamic_viscosity_water
slist *= td
t = td * 10
Rep =100#1
print(factor_drag_white1991(Rep))
ax.plot(slist,Basset_history_kernel_function(slist,t,diameter,dynamic_viscosity_water),'-',color='grey')
ax.plot(slist, Mei_history_kernel_function(slist, t, diameter, dynamic_viscosity_water, Rep, cs['Mei']['c1'], cs['Mei']['c2']),'--',color='orange')
ax.plot(slist, Mei_history_kernel_function(slist, t, diameter, dynamic_viscosity_water, Rep, cs['Dorgan']['c1'], cs['Dorgan']['c2']),'--',color='purple')
ax.plot(slist, Mei_history_kernel_function(slist, t, diameter, dynamic_viscosity_water, Rep, cs['Kim']['c1'], cs['Kim']['c2']),'--',color='cyan')

ax.plot(slist,Talaei_history_kernel_function(slist,t,diameter,dynamic_viscosity_water,Rep),'-.',color='navy')
ax.plot(slist,Talaei_history_kernel_function_original(slist,t,diameter,dynamic_viscosity_water,Rep),':',color='firebrick') # correct calculation
ax.set_yscale('log')

# Calculate history term for simulation data
Use force as unit as I am not sure about how to handle the history term in the slow manifold MR derivation.

In [ ]:
# need to find correct timescales
tau_diff = diffusion_time(diameter,kinematic_viscosity_water) # in sec
print(f'tau_diff = {tau_diff:.2f} s') 
for Rep in Replist:
    if(Rep==0):
        continue
    tau_H = history_timescale(Rep,tau_diff) # in sec
    print(f'tau_H = {tau_H:.2f} s') #7.5 min

# calculate basset history term for high time resolution data
We use data that ran for 24 h with output every 5 min to calculate
the basset history kernel and term and invastigate its sensitivity
to total timewindow and timesteps used. 

In [ ]:
# /storage/shared/oceanparcels/output_data/data_Meike/MR_advection/NWES/test_history_term/inertial_SM_Rep_constant/NWES_start2023_09_01_end2023_10_01_RK4__Rep_0100_B0680_tau2994_anti_beaching_cor_True_gradient_True.zarr.



In [ ]:
# settings
starttime = datetime(2023,9,1,0,0,0)
runtime = timedelta(days =2)
endtime = starttime + runtime
loc = 'NWES'
land_handling = 'anti_beaching'
coriolis = True
B = 0.68
tau = 2994.76
Replist = [0,10,100,450,1000]
starttimes = [datetime(2023, 9, 1, 0, 0, 0, 0)]
nparticles = 88347 # 52511
chunck_time = 100
coriolis = True
gradient = True
displacement=300

# nparticles_select =1000
# chunck_time=100

base_directory = '/storage/shared/oceanparcels/output_data/data_Meike/MR_advection/NWES/test_history_term/'
basefile_Rep_constant = (base_directory + '{particle_type}/{loc}_'
                 'start{y_s:04d}_{m_s:02d}_{d_s:02d}_'
                 'end{y_e:04d}_{m_e:02d}_{d_e:02d}_RK4_'
                 '_Rep_{Rep:04d}_B{B:04d}_tau{tau:04d}_{land_handling}_cor_{coriolis}_gradient_{gradient}.zarr')

basefile_Rep_drag = (base_directory + '{particle_type}/{loc}_start{y_s:04d}_{m_s:02d}_{d_s:02d}'
                 '_end{y_e:04d}_{m_e:02d}_{d_e:02d}_RK4_B{B:04d}_tau{tau:04d}_{land_handling}_cor_{coriolis}_gradient_{gradient}.zarr')

basefiles={
           'inertial_Rep_constant':basefile_Rep_constant,
           'inertial_SM_Rep_constant':basefile_Rep_constant, 
           'inertial_SM_drag_Rep':basefile_Rep_drag,
           'inertial_drag_Rep':basefile_Rep_drag}

particle_types = ['inertial_drag_Rep','inertial_Rep_constant']#,'inertial_Rep_constant'] # 
names = {'inertial_Rep_constant':'MR',
         'inertial_SM_Rep_constant':'SM MR', 
         'inertial_SM_drag_Rep':'SM MR flexible Re$_p$',
         'inertial_drag_Rep':'MR flexible Re$_p$'}



In [ ]:
# read in data 
dt= 5 * 60
data ={}
for pt in particle_types:
    data[pt]={}
    for coriolis in [True]:
        data[pt][coriolis]={}
        if(pt in ('inertial_SM_Rep_constant','inertial_Rep_constant')):
            for Rep in Replist:
                data[pt][coriolis][Rep]={}
        else:
            data[pt][coriolis][None]={}

for pt in particle_types:
    for coriolis in [True]:
        for starttime in starttimes:
            # print(starttime)
            endtime = starttime + runtime 
            date = f'{starttime.year:04d}/{starttime.month:02d}'
            
            if(pt in ('inertial_SM_Rep_constant','inertial_Rep_constant')):
                
                for Rep in Replist:
                    file = basefiles[pt].format(loc=loc,
                                                y_s=starttime.year,
                                                m_s=starttime.month,
                                                d_s=starttime.day,
                                                y_e=endtime.year,
                                                m_e=endtime.month,
                                                d_e=endtime.day,
                                                B = int(B * 1000), 
                                                tau = int(tau ),
                                                land_handling = land_handling, 
                                                coriolis = coriolis,
                                                gradient = gradient,
                                                particle_type = pt,
                                                Rep = Rep)
                    # print(file)
                    ds = xr.open_dataset(file,
                                        engine='zarr',
                                        chunks={'trajectory':nparticles, 'obs':chunck_time},
                                        drop_variables=['B','tau','z'],
                                        decode_times=False) #,decode_cf=False)

                    Uslip = np.sqrt(ds.uslip**2 + ds.vslip**2) 
                    ds = ds.assign(Uslip = Uslip)
                    dUslipdt = derivative_backward(ds.Uslip,dt)
                    ds = ds.assign(dUslipdt = dUslipdt)
                    duslipdt = derivative_backward(ds.uslip,dt)
                    ds = ds.assign(duslipdt = duslipdt)
                    dvslipdt = derivative_backward(ds.vslip,dt)
                    ds = ds.assign(dvslipdt = dvslipdt)
                    data[pt][coriolis][Rep][date]= ds 
            else:
                # print (pt)
                file = basefiles[pt].format(loc=loc,
                                            y_s=starttime.year,
                                            m_s=starttime.month,
                                            d_s=starttime.day,
                                            y_e=endtime.year,
                                            m_e=endtime.month,
                                            d_e=endtime.day,
                                            land_handling = land_handling, 
                                            coriolis = coriolis,
                                            gradient = gradient,
                                            d=displacement,
                                            B = int(B * 1000), 
                                            tau = int(tau ),
                                            particle_type = pt)
                ds = xr.open_dataset(file,
                                    engine='zarr',
                                    chunks={'trajectory':nparticles, 'obs':chunck_time},
                                    drop_variables=['z'],
                                    decode_times=False) #,decode_cf=False)

                Uslip = np.sqrt(ds.uslip**2 + ds.vslip**2) 
                ds = ds.assign(Uslip = Uslip)
                dUslipdt = derivative_backward(ds.Uslip,dt)
                ds = ds.assign(dUslipdt = dUslipdt)
                duslipdt = derivative_backward(ds.uslip,dt)
                ds = ds.assign(duslipdt = duslipdt)
                dvslipdt = derivative_backward(ds.vslip,dt)
                ds = ds.assign(dvslipdt = dvslipdt)

                # Rep_measured = Re_particle(ds.Uslip,diameter, kinematic_viscosity_water)
                # ds = ds.assign(Rep_measured = Rep_measured)
                data[pt][coriolis][None][date]= ds 

            
          
  
        



In [ ]:
# plot raw data slip velocity, cubicspline fit and time derivative calcuated using the fit toghether

id = 2
dt_write =10
Tmax = 12 * 47
nav = 12
fig,ax=plt.subplots()
ax2=ax.twinx()
markers=['-','--','-','--','-','--','-','--','-','--','-','--']
for pt, marker in zip(particle_types,markers):

    if(pt in ('inertial_SM_Rep_constant','inertial_Rep_constant')):
        for Rep, color in zip(Replist,Magmalist):

            ax.plot(data[pt][coriolis][Rep][date].time[id,10:]/3600,data[pt][coriolis][Rep][date].uslip[id,10:],'.',color=color,alpha=0.4)
            time = data[pt][coriolis][Rep][date].time[id,nav+1:Tmax].values-dt * nav / 2
            av_uslip = calc_tidal_av(data[pt][coriolis][Rep][date].uslip,nav)

            y = av_uslip[id,nav+1:Tmax].values

            spl = CubicSpline(time, y)
            der = spl.derivative(1)
           
            ax2.plot(time/3600,der(time),'--',color=color)
            
            ax.plot(time/3600,spl(time), '-',color=color)
    else:
        ax.plot(data[pt][coriolis][None][date].time[id,10:]/3600,data[pt][coriolis][None][date].uslip[id,10:],'.',color='cornflowerblue',alpha=0.4)
        time = data[pt][coriolis][None][date].time[id,nav+1:Tmax].values-dt * nav / 2
        av_uslip = calc_tidal_av(data[pt][coriolis][None][date].uslip,nav)
        y = av_uslip[id,nav+1:Tmax].values
    
        spl = CubicSpline(time, y)
        der = spl.derivative(1)
        
        ax2.plot(time/3600,der(time),'--',color='cornflowerblue')
        
        ax.plot(time/3600,spl(time), '-',color='cornflowerblue')




ax.set_xlabel('time [hours]')
ax.set_ylabel('$u_{\\mathrm{slip}}$ [m/s]')
ax2.set_ylabel('$du_{\\mathrm{slip}}/dt$ [m/s$^2$]')
ax.ticklabel_format(axis='y',style='sci',scilimits=(0,0))

ax.legend(['measured','smoothed'],loc=(0.7,0.85),fontsize=16)
ax2.legend(['derivative'],loc=(0.7,0.8),fontsize=16)



In [ ]:
## quick hack to fix label without recalculating data

# fig, ax  = plt.subplots()
# Replist = [0,10,100,450,1000]
# legend = ['flexible Re$_p^{\\mathrm{In}}$']
# ax.plot(0.001,20,'--',color='cornflowerblue')
# for Rep,color in zip(Replist,Magmalist):
#     ax.plot(0.001,20,'-.',color=color)
#     legend.append('Re$_p^{\\mathrm{In}}$='+f'{Rep}')

# ax.legend(legend,fontsize=18,handlelength=1.5)
# ax.set_xscale('log')
# ax.set_xlim(10e-5,10e1)
# ax.set_ylim(-2,41)
# ax.set_ylabel('PDF')
# ax.set_xlabel('history term / drag term')

In [ ]:
# calculate history term with mei history kernel

fig,ax=plt.subplots()
# ax2=ax.twinx()
id = 2
Tmax = 12 * 47
nav = 12#12
nresample = 3
dt =300
dt_resample = dt/nresample
tau_diff = diffusion_time(diameter,kinematic_viscosity_water) # in sec
# loop over all particles
for pt in particle_types:
    # if rep is constant loop over rep values
    if(pt == 'inertial_Rep_constant'):
        
        for Rep, color in zip(Replist,Magmalist):
            # select trajectory
            ds = data[pt][coriolis][Rep][date].isel(trajectory=id)
            # calculate window averages
            da_av_uslip = calc_tidal_av(ds.uslip,nav)
            da_av_vslip = calc_tidal_av(ds.vslip,nav)
            # make reshifted time array
            time = ds.time[nav+1:Tmax].values-dt *nav /2

            # make reshifted, resampled time array
            time_resample = np.arange(time[0],time[-1],dt_resample)

            # load data to array to put in to cubicspline
            av_uslip =  da_av_uslip[nav+1:Tmax].values
            av_vslip =  da_av_vslip[nav+1:Tmax].values

            # calculate cubicspline for data
            spl_uslip = CubicSpline(time, av_uslip)
            spl_vslip = CubicSpline(time, av_vslip)

            # calculate derivatives from the cubicplines
            der_uslip = spl_uslip.derivative(1)
            der_vslip = spl_vslip.derivative(1)

            # sample derivative into array using the resampled time
            array_duslipdt = der_uslip(time_resample)
            array_dvslipdt = der_vslip(time_resample)


            # calculate timescale, for rep = 0 officially infinite so ure rep = 1 instead
            if(Rep == 0 ):
                tau_H= history_timescale(1,tau_diff)
            else:
                tau_H = 1* history_timescale(Rep,tau_diff) # in sec
            tau_H = 4*3600 # or use constant time for all values? -

            # define number of timesteps when calculating history term
            Nwindow = int(tau_H/dt_resample)

            # define maximum index number for calculating history term
            Nmax = int((Tmax-nav-1)*nresample)


            # define timelist given the maximum index number
            Tlist = np.arange(0,Nmax,1)*dt_resample

            #create empty list to save history force into
            F_H_Nwindow = []

            # create emptly list to save time values into
            tvalues = []
        
            # loop over original timesteps to calculate history term
            for N in range (Nwindow+nresample,Nmax-nresample,nresample):
                T=Tlist[N]
                # calculate x and y component history force
                F_H_x =History_Force_Hinsberg_Mei_kernel(array_duslipdt[N-Nwindow-1:N+1],T,cs['Dorgan']['c1'],cs['Dorgan']['c2'],Rep,Nwindow,dt_resample,diameter,kinematic_viscosity_water)
                F_H_y =History_Force_Hinsberg_Mei_kernel(array_dvslipdt[N-Nwindow-1:N+1],T,cs['Dorgan']['c1'],cs['Dorgan']['c2'],Rep,Nwindow,dt_resample,diameter,kinematic_viscosity_water)
                # calculate magnitude of force
                F_H = np.sqrt(F_H_x**2+F_H_y**2)
               
                # write magnitude to list
                F_H_Nwindow.append(F_H)
                # write timestep to lists
                tvalues.append(T)

            # make np array from lists
            F_B_Nwindow = np.asarray(F_B_Nwindow)
            tvalues = np.asarray(tvalues)
            
            # calculate magnitude of slip velocity
            F_Uslip = np.sqrt(spl_uslip(tvalues)**2 + spl_vslip(tvalues)**2)*factor_drag_white1991(Rep)
            
            # plot ratio history term and slip velocity
            ax.plot(tvalues/3600,F_H_Nwindow/F_Uslip,'-.',color=color)

    else:
        # select trajectory
        ds = data[pt][coriolis][None][date].isel(trajectory=id)
        # calculate window averages
        da_av_uslip = calc_tidal_av(ds.uslip,nav)
        da_av_vslip = calc_tidal_av(ds.vslip,nav)
        # make reshifted time array
        time = ds.time[nav+1:Tmax].values-dt *nav /2
        # make reshifted, resampled time array
        time_resample = np.arange(time[0],time[-1],dt_resample)
        # load data to array to put in to cubicspline
        av_uslip =  da_av_uslip[nav+1:Tmax].values
        av_vslip =  da_av_vslip[nav+1:Tmax].values
        # calculate cubicspline for data
        spl_uslip = CubicSpline(time, av_uslip)
        spl_vslip = CubicSpline(time, av_vslip)
        # calculate derivatives from the cubicplines
        der_uslip = spl_uslip.derivative(1)
        der_vslip = spl_vslip.derivative(1)
        # sample derivative into array using the resampled time
        array_duslipdt = der_uslip(time_resample)
        array_dvslipdt = der_vslip(time_resample)

        # calculate particle reynolds number with resampled data
        rep_measured = Re_particle(np.sqrt(spl_uslip(time_resample)**2+spl_vslip(time_resample)**2),diameter,kinematic_viscosity_water)
        # calculate mean particlereynolds number
        rep_mean = np.mean(rep_measured)
        print(rep_mean)
        # tau_H= history_timescale(rep_mean,tau_diff)
        tau_H = 4*3600 
        # define number of timesteps when calculating history term
        Nwindow = int(tau_H/dt_resample)

        # define maximum index number for calculating history term
        Nmax = int((Tmax-nav-1)*nresample)
        
        # define timelist given the maximum index number
        Tlist = np.arange(0,Nmax,1)*dt_resample

         #create empty list to save history force into
        F_H_Nwindow=[]

        # create emptly list to save time values into
        tvalues=[]

        # loop over original timesteps to calculate history term
        for N in range (Nwindow+nresample,Nmax-nresample,nresample):
            T=Tlist[N]
            # calculate x and y component history force
            F_H_x =History_Force_Hinsberg_Mei_kernel(array_duslipdt[N-Nwindow-1:N+1],T,cs['Dorgan']['c1'],cs['Dorgan']['c2'],rep_measured[N-Nwindow-1:N+1],Nwindow,dt_resample,diameter,kinematic_viscosity_water)
            F_H_y =History_Force_Hinsberg_Mei_kernel(array_dvslipdt[N-Nwindow-1:N+1],T,cs['Dorgan']['c1'],cs['Dorgan']['c2'],rep_measured[N-Nwindow-1:N+1],Nwindow,dt_resample,diameter,kinematic_viscosity_water)
            # calculate magnitude of force
            F_H =np.sqrt(F_H_x**2+F_H_y**2)
           
            # write magnitude to list
            F_H_Nwindow.append(F_H)
            # write timestep to lists
            tvalues.append(T)

        # make np array from lists
        F_H_Nwindow = np.asarray(F_H_Nwindow)
        tvalues = np.asarray(tvalues)

        # calculate magnitude of slip velocity
        F_Uslip = np.sqrt(spl_uslip(tvalues)**2 + spl_vslip(tvalues)**2)*factor_drag_white1991(rep_measured[Nwindow+nresample:Nmax-nresample:nresample])
        # plot ratio history term and slip velocity
        ax.plot(tvalues/3600,F_H_Nwindow/F_Uslip,'-',color='cornflowerblue')

        Tlist = np.arange(0,Nmax,1)*dt_resample



ax.legend(['history term','slip velocity term'])
ax.set_xlabel('time [hours]')
ax.set_ylabel('acceleration [m/s$^2$]')
ax.set_yscale('log')


In [ ]:
obs_vals = np.linspace(0,100,11)
print(obs_vals)
n_resample = 10
obs_resampled = np.linspace(obs_vals.min(), obs_vals.max(), num=(len(obs_vals) - 1) * n_resample + 1)
print(obs_resampled)

In [ ]:
# calculate history term with mei history kernel loop over trajectories

fig,ax=plt.subplots()
# ax2=ax.twinx()

Tmax = 12 * 47
nav = 12#12
nresample = 1
dt =300
dt_resample = dt/nresample
tau_diff = diffusion_time(diameter,kinematic_viscosity_water) # in sec
# loop over all particles
for pt in particle_types:
    # if rep is constant loop over rep values
    if(pt == 'inertial_Rep_constant'):
        
        for Rep, color in zip(Replist,Magmalist):
            print(Rep)
            ratio_array = []
            # loop over trajectories
            for id in range(0,5,1): #52511
                ds = data[pt][coriolis][Rep][date].isel(trajectory=id)
                 # calculate window averages
                da_av_uslip = calc_tidal_av(ds.uslip,nav)
                da_av_vslip = calc_tidal_av(ds.vslip,nav)
                # make reshifted time array
                time = ds.time[nav+1:Tmax].values-dt *nav /2
                if(np.isnan(time[-1])):
                    continue
                # make reshifted, resampled time array
                time_resample = np.arange(time[0],time[-1],dt_resample)

                # load data to array to put in to cubicspline
                av_uslip =  da_av_uslip[nav+1:Tmax].values
                av_vslip =  da_av_vslip[nav+1:Tmax].values

                # calculate cubicspline for data
                spl_uslip = CubicSpline(time, av_uslip)
                spl_vslip = CubicSpline(time, av_vslip)

                # calculate derivatives from the cubicplines
                der_uslip = spl_uslip.derivative(1)
                der_vslip = spl_vslip.derivative(1)

                # sample derivative into array using the resampled time
                array_duslipdt = der_uslip(time_resample)
                array_dvslipdt = der_vslip(time_resample)


                # # calculate timescale, for rep = 0 officially infinite so ure rep = 1 instead
                # if(Rep == 0 ):
                #     tau_H= history_timescale(1,tau_diff)
                # else:
                #     tau_H = 1* history_timescale(Rep,tau_diff) # in sec
                tau_H = 1*3600 # or use constant time for all values? -

                # define number of timesteps when calculating history term
                Nwindow = int(tau_H/dt_resample)

                # define maximum index number for calculating history term
                Nmax = int((Tmax-nav-1)*nresample)


                # define timelist given the maximum index number
                Tlist = np.arange(0,Nmax,1)*dt_resample

                #create empty list to save history force into
                F_H_Nwindow = []

                # create emptly list to save time values into
       
            
                # loop over original timesteps to calculate history term
                for N in range (Nwindow+nresample,Nmax-nresample,nresample):
          
                    # calculate x and y component history force
                    F_H_x =History_Force_Hinsberg_Mei_kernel(array_duslipdt[N-Nwindow-1:N+1],Tlist[N],cs['Dorgan']['c1'],cs['Dorgan']['c2'],Rep,Nwindow,dt_resample,diameter,kinematic_viscosity_water)
                    F_H_y =History_Force_Hinsberg_Mei_kernel(array_dvslipdt[N-Nwindow-1:N+1],Tlist[N],cs['Dorgan']['c1'],cs['Dorgan']['c2'],Rep,Nwindow,dt_resample,diameter,kinematic_viscosity_water)
                    # calculate magnitude of force
                    F_H = np.sqrt(F_H_x**2+F_H_y**2)
                
                    # write magnitude to list
                    F_H_Nwindow.append(F_H)
                    # write timestep to lists
      

                # make np array from lists
                F_H_Nwindow = np.asarray(F_H_Nwindow)
       
                tvalues = Tlist[Nwindow+nresample:Nmax-nresample:nresample]
                # calculate magnitude of slip velocity
                F_Uslip = np.sqrt(spl_uslip(tvalues)**2 + spl_vslip(tvalues)**2)*factor_drag_white1991(Rep)
                
                ratio_array.append(F_H_Nwindow/F_Uslip)
            ratio_array=np.asarray(ratio_array).flatten()
            ratio_array=ratio_array[~np.isnan(ratio_array)]  
            
            bins, pdf = make_lognormal_PDF(ratio_array,nbins=100,norm=True,vmin=1e-3,vmax=1e1);
            ax.plot(bins,pdf,'-.',color=color)
    else:
        # loop over trajectories
        ratio_array = []
        for id in range(0,5,1): #52511

            ds = data[pt][coriolis][None][date].isel(trajectory=id)
            # calculate window averages
            da_av_uslip = calc_tidal_av(ds.uslip,nav)
            da_av_vslip = calc_tidal_av(ds.vslip,nav)
            # make reshifted time array
            time = ds.time[nav+1:Tmax].values-dt *nav /2
            if(np.isnan(time[-1])):
                continue
            # print(time[-1])
            # make reshifted, resampled time array
            time_resample = np.arange(time[0],time[-1],dt_resample)
            # load data to array to put in to cubicspline
            av_uslip =  da_av_uslip[nav+1:Tmax].values
            av_vslip =  da_av_vslip[nav+1:Tmax].values
            # calculate cubicspline for data
            spl_uslip = CubicSpline(time, av_uslip)
            spl_vslip = CubicSpline(time, av_vslip)
            # calculate derivatives from the cubicplines
            der_uslip = spl_uslip.derivative(1)
            der_vslip = spl_vslip.derivative(1)
            # sample derivative into array using the resampled time
            array_duslipdt = der_uslip(time_resample)
            array_dvslipdt = der_vslip(time_resample)

            # calculate particle reynolds number with resampled data
            rep_measured = Re_particle(np.sqrt(spl_uslip(time_resample)**2+spl_vslip(time_resample)**2),diameter,kinematic_viscosity_water)
            # calculate mean particlereynolds number
            rep_mean = np.mean(rep_measured)
            # tau_H= history_timescale(rep_mean,tau_diff)
            tau_H = 1*3600 
            # define number of timesteps when calculating history term
            Nwindow = int(tau_H/dt_resample)

            # define maximum index number for calculating history term
            Nmax = int((Tmax-nav-1)*nresample)
            
            # define timelist given the maximum index number
            Tlist = np.arange(0,Nmax,1)*dt_resample

            #create empty list to save history force into
            F_H_Nwindow=[]

            # create emptly list to save time values into
 

            # loop over original timesteps to calculate history term
            for N in range (Nwindow+nresample,Nmax-nresample,nresample):

                # calculate x and y component history force
                F_H_x =History_Force_Hinsberg_Mei_kernel(array_duslipdt[N-Nwindow-1:N+1],Tlist[N],cs['Dorgan']['c1'],cs['Dorgan']['c2'],rep_measured[N-Nwindow-1:N+1],Nwindow,dt_resample,diameter,kinematic_viscosity_water)
                F_H_y =History_Force_Hinsberg_Mei_kernel(array_dvslipdt[N-Nwindow-1:N+1],Tlist[N],cs['Dorgan']['c1'],cs['Dorgan']['c2'],rep_measured[N-Nwindow-1:N+1],Nwindow,dt_resample,diameter,kinematic_viscosity_water)
                # calculate magnitude of force
                F_H =np.sqrt(F_H_x**2+F_H_y**2)
            
                # write magnitude to list
                F_H_Nwindow.append(F_H)
                # write timestep to lists
   

            # make np array from lists
            F_H_Nwindow = np.asarray(F_H_Nwindow)
            tvalues = Tlist[Nwindow+nresample:Nmax-nresample:nresample]

            # calculate magnitude of slip velocity
            F_Uslip = np.sqrt(spl_uslip(tvalues)**2 + spl_vslip(tvalues)**2)*factor_drag_white1991(rep_measured[Nwindow+nresample:Nmax-nresample:nresample])
            # plot ratio history term and slip velocity
            ratio_array.append(F_H_Nwindow/F_Uslip)

        ratio_array=np.asarray(ratio_array).flatten()
        ratio_array=ratio_array[~np.isnan(ratio_array)]    
        bins, pdf = make_lognormal_PDF(ratio_array,nbins=100,norm=True,vmin=1e-3,vmax=1e1);
        ax.plot(bins,pdf,'--',color='cornflowerblue')



ax.legend(['history term','slip velocity term'])
ax.set_xlabel('Fhistory/Fslip')
ax.set_ylabel('PDF')
# ax.set_yscale('log')
ax.set_xscale('log')


In [ ]:
# Define a function that works on 1D velocity arrays
def spline_derivative(v, t):
    cs = CubicSpline(t, v)
    return cs.derivative()(t)

def spline(v, t):
    cs = CubicSpline(t, v)
    return cs(t)


def make_windowed_da(da: xr.DataArray, nwindow: int) -> xr.DataArray:
    traj_dim = 'trajectory'
    obs_dim = 'obs'

    # Pad with zeros on the left (before obs=0)
    padded = np.pad(da.values, ((0, 0), (nwindow - 1, 0)), mode='constant')

    # Create windowed array
    result = np.lib.stride_tricks.sliding_window_view(padded, window_shape=nwindow, axis=1)

    # result shape: (trajectory, obs, nwindow)
    coords = {
        'trajectory': da.coords[traj_dim],
        'obs': da.coords[obs_dim],
        'nwindow': np.arange(nwindow)
    }

    return xr.DataArray(result, dims=(traj_dim, obs_dim, 'nwindow'), coords=coords)

import dask.array as daskarray

def make_windowed_da_dask(da_xr: xr.DataArray, nwindow: int) -> xr.DataArray:
    traj_dim = 'trajectory'
    obs_dim = 'obs'
    
    darr = da_xr.data  # this is a dask array
    
    # Pad along obs axis (axis=1)
    pad_width = ((0, 0), (nwindow - 1, 0))  # pad only obs axis on left
    padded = daskarray.pad(darr, pad_width, mode='constant', constant_values=0)
    
    # Use dask's sliding_window_view
    result = daskarray.lib.stride_tricks.sliding_window_view(padded, window_shape=nwindow, axis=1)
    
    # result shape: (trajectory, obs, nwindow)
    coords = {
        traj_dim: da_xr.coords[traj_dim],
        obs_dim: da_xr.coords[obs_dim],
        'nwindow': np.arange(nwindow)
    }
    
    return xr.DataArray(result, dims=(traj_dim, obs_dim, 'nwindow'), coords=coords)

def f_Mei1992_unit_vectorized(dudt, t, c1, c2, Rep, h, tau_diff):
    """
    dudt: array of shape (nwindow,)
    t: scalar
    Rep: scalar or 1D if needed
    Returns: array of shape (nwindow,)
    """
    n = np.arange(len(dudt))
    delta_t = t - h * n

    kernel = (
        ((delta_t**3 * 12 * np.pi) / (tau_diff * t**3)) ** 0.5
        * Rep**3 / (16 * np.pi * (0.75 + c2 * Rep)**3)
    )
    factor = (1 + kernel ** (1 / c1)) ** (-c1)

    return dudt * factor

from numba import njit

@njit
def f_Mei1992_unit_vectorized_numba(dudt, t, c1, c2, Rep, h, tau_diff):
    nwindow = len(dudt)
    result = np.empty(nwindow, dtype=dudt.dtype)
    for i in range(nwindow):
        delta_t = t - h * i
        if delta_t <= 0:
            result[i] = 0.0
            continue
        kernel = (
            ((delta_t**3 * 12 * np.pi) / (tau_diff * t**3)) ** 0.5
            * Rep**3 / (16 * np.pi * (0.75 + c2 * Rep)**3)
        )
        factor = (1 + kernel ** (1 / c1)) ** (-c1)
        result[i] = dudt[i] * factor
    return result


@njit
def f_Mei1992_unit_vectorized_numba_Rep_array(dudt, Rep_arr, t, c1, c2, h, tau_diff):
    """
    dudt: 1D array (length = nwindow)
    Rep_arr: 1D array (same shape as dudt)
    t: scalar (time)
    Returns: array of shape (nwindow,)
    """
    nwindow = len(dudt)
    result = np.empty(nwindow, dtype=dudt.dtype)
    
    for i in range(nwindow):
        delta_t = t - h * i
        if delta_t <= 0:
            result[i] = 0.0
            continue
        
        Rep = Rep_arr[i]
        kernel = (
            ((delta_t**3 * 12 * np.pi) / (tau_diff * t**3)) ** 0.5
            * Rep**3 / (16 * np.pi * (0.75 + c2 * Rep)**3)
        )
        factor = (1 + kernel ** (1 / c1)) ** (-c1)
        result[i] = dudt[i] * factor
    
    return result

def Hinsberg_vectorized(f, N, h):
    """
    f: 1D array with length N+1 (assumed)
    Returns scalar
    """
    K0 = 4 / 3 * f[N] * np.sqrt(h)
    KN = (
        f[0]
        * np.sqrt(h)
        * (N - 4 / 3)
        / ((N - 1) * np.sqrt(N - 1) + (N - 3 / 2) * np.sqrt(N))
    )
    # Guard against small N
    if N < 2:
        return K0 + KN

    k = np.arange(1, N)
    KSUM = np.sum(
        np.sqrt(h)
        * f[N - k]
        * (
            (k + 4 / 3) / ((k + 1) ** (3 / 2) + (k + 3 / 2) * np.sqrt(k))
            + (k - 4 / 3) / ((k - 1) ** (3 / 2) + (k - 3 / 2) * np.sqrt(k))
        )
    )
    return K0 + KN + KSUM

@njit
def Hinsberg_vectorized_numba(f, N, h):
    """
    f: 1D array with length N+1 (assumed)
    Returns scalar
    """
    K0 = (4.0 / 3.0) * f[N] * np.sqrt(h)

    # Guard against division by zero or small N
    if N < 1:
        return K0  # No KN or KSUM if N < 1

    denom_KN = (N - 1) * np.sqrt(N - 1) + (N - 1.5) * np.sqrt(N)
    if denom_KN == 0:
        KN = 0.0
    else:
        KN = f[0] * np.sqrt(h) * (N - 4.0 / 3.0) / denom_KN

    if N < 2:
        return K0 + KN

    KSUM = 0.0
    for k in range(1, N):
        denom1 = (k + 1)**1.5 + (k + 1.5)*np.sqrt(k)
        denom2 = (k - 1)**1.5 + (k - 1.5)*np.sqrt(k)

        term1 = 0.0
        term2 = 0.0
        if denom1 != 0:
            term1 = (k + 4.0 / 3.0) / denom1
        if denom2 != 0:
            term2 = (k - 4.0 / 3.0) / denom2

        KSUM += np.sqrt(h) * f[N - k] * (term1 + term2)

    return K0 + KN + KSUM



In [ ]:
data_history_slip={}
for pt in particle_types:
    data_history_slip[pt]={}



# now make avearge over all trajectories
id = 2
Tmax = 12 * 47
nav = 12#12
pt = 'inertial_Rep_constant'
nresample = 30
dt =300
tau_H =  1*3600
Nwindow = int(tau_H/dt)
Nmax = int(Tmax-nav-1)
dt_resample = dt/nresample
Tlist = np.arange(0,Nmax,1)*dt
tau_diff = diffusion_time(diameter,kinematic_viscosity_water) # in sec

fig, ax = plt.subplots()

for pt in particle_types:
    # fig,ax=plt.subplots()
    if(pt == 'inertial_Rep_constant'):
        
        
        for Rep, color in zip(Replist,Magmalist):
            ds = data[pt][coriolis][Rep][date]
            # ds = ds.isel(trajectory=slice(0,10))
            ds_check = ds.time.isel(obs=Tmax)
            filter = np.argwhere(~np.isnan(ds_check.values)).flatten()
            ds_sel=ds.isel(trajectory=filter)
            da_av_uslip = calc_tidal_av(ds_sel.uslip,nav)
            da_av_vslip = calc_tidal_av(ds_sel.vslip,nav)
            # u
            da_av_uslip  = da_av_uslip.isel(obs=slice(nav+1,Tmax))
            da_av_uslip = da_av_uslip.chunk({'trajectory':1000,'obs':-1})
            # v
            da_av_vslip  = da_av_vslip.isel(obs=slice(nav+1,Tmax))
            da_av_vslip = da_av_vslip.chunk({'trajectory':1000,'obs':-1})
            
            # define time array
            time_da = ds.time.isel(trajectory=2, obs=slice(nav + 1, Tmax)) - dt * nav / 2
            time_da = time_da.chunk(dict(obs=-1))

            # # calculate derivative using ufunc
            da_der_uslip = xr.apply_ufunc( spline_derivative, da_av_uslip, time_da,
                input_core_dims=[["obs"], ["obs"]], output_core_dims=[["obs"]],
                vectorize=True, dask="parallelized", output_dtypes=[float],join="override")
            da_der_vslip = xr.apply_ufunc( spline_derivative, da_av_vslip, time_da,
                input_core_dims=[["obs"], ["obs"]], output_core_dims=[["obs"]],
                vectorize=True, dask="parallelized", output_dtypes=[float],join="override")
            
            #calculate slip velocties and force 
            da_uslip = xr.apply_ufunc( spline, da_av_uslip, time_da,
                input_core_dims=[["obs"], ["obs"]], output_core_dims=[["obs"]],
                vectorize=True, dask="parallelized", output_dtypes=[float],join="override")
            da_vslip = xr.apply_ufunc( spline, da_av_vslip, time_da,
                input_core_dims=[["obs"], ["obs"]], output_core_dims=[["obs"]],
                vectorize=True, dask="parallelized", output_dtypes=[float],join="override")
            
            da_Uslip = np.sqrt(da_uslip**2 + da_vslip**2)

            da_rep = Re_particle(da_Uslip,diameter,kinematic_viscosity_water)
 
            da_Fslip = da_Uslip*factor_drag_white1991(Rep)
     

            da_der_uslip_nwindow = make_windowed_da_dask(da_der_uslip,Nwindow+1)
            da_der_vslip_nwindow = make_windowed_da_dask(da_der_vslip,Nwindow+1)
            da_u_f_mei1992 = xr.apply_ufunc(
                f_Mei1992_unit_vectorized_numba,
                da_der_uslip_nwindow,   # (trajectory, obs, nwindow)
                time_da,                # (obs,) or (trajectory, obs)
                kwargs=dict(c1=cs['Dorgan']['c1'],c2=cs['Dorgan']['c2'], Rep=Rep, h=dt, tau_diff=tau_diff),
                input_core_dims=[["nwindow"], []],
                output_core_dims=[["nwindow"]],
                vectorize=True,
                dask="parallelized",
                output_dtypes=[float],
            )
            da_v_f_mei1992 = xr.apply_ufunc(
                f_Mei1992_unit_vectorized_numba,
                da_der_vslip_nwindow,   # (trajectory, obs, nwindow)
                time_da,                # (obs,) or (trajectory, obs)
                kwargs=dict(c1=cs['Dorgan']['c1'],c2=cs['Dorgan']['c2'], Rep=Rep, h=dt, tau_diff=tau_diff),
                input_core_dims=[["nwindow"], []],
                output_core_dims=[["nwindow"]],
                vectorize=True,
                dask="parallelized",
                output_dtypes=[float],
            )

            da_u_hinsberg = xr.apply_ufunc(
                Hinsberg_vectorized_numba,
                da_u_f_mei1992,
                kwargs=dict(N=Nwindow, h=dt),
                input_core_dims=[["nwindow"]],
                output_core_dims=[[]],
                vectorize=True,
                dask="parallelized",
                output_dtypes=[float],
            )
            da_v_hinsberg = xr.apply_ufunc(
                Hinsberg_vectorized_numba,
                da_v_f_mei1992,
                kwargs=dict(N=Nwindow, h=dt),
                input_core_dims=[["nwindow"]],
                output_core_dims=[[]],
                vectorize=True,
                dask="parallelized",
                output_dtypes=[float],
            )
            da_u_history = np.sqrt(tau_diff / (4 * np.pi)) * da_u_hinsberg
            ds_v_history = np.sqrt(tau_diff / (4 * np.pi)) * da_v_hinsberg
            da_F_h = np.sqrt(da_u_history**2 + ds_v_history**2)
            da_ratio = da_F_h/da_Fslip
            array = da_ratio.values.flatten()
            array = array[~np.isnan(array)]
            data_history_slip[pt][Rep]=array
            bins,pdf = make_lognormal_PDF(array,nbins=100,norm=True,vmin=1e-4,vmax=1e2)
            ax.plot(bins,pdf,'-.',color=color)
    else:
        ds = data[pt][coriolis][None][date]
        # ds = ds.isel(trajectory=slice(0,10))
        ds_check = ds.time.isel(obs=Tmax)
        filter = np.argwhere(~np.isnan(ds_check.values)).flatten()
        ds_sel=ds.isel(trajectory=filter)
        da_av_uslip = calc_tidal_av(ds_sel.uslip,nav)
        da_av_vslip = calc_tidal_av(ds_sel.vslip,nav)
        # u
        da_av_uslip  = da_av_uslip.isel(obs=slice(nav+1,Tmax))
        da_av_uslip = da_av_uslip.chunk({'trajectory':1000,'obs':-1})
        # v
        da_av_vslip  = da_av_vslip.isel(obs=slice(nav+1,Tmax))
        da_av_vslip = da_av_vslip.chunk({'trajectory':1000,'obs':-1})
        
        # define time array
        time_da = ds.time.isel(trajectory=2, obs=slice(nav + 1, Tmax)) - dt * nav / 2
        time_da = time_da.chunk(dict(obs=-1))

        # # calculate derivative using ufunc
        da_der_uslip = xr.apply_ufunc( spline_derivative, da_av_uslip, time_da,
            input_core_dims=[["obs"], ["obs"]], output_core_dims=[["obs"]],
            vectorize=True, dask="parallelized", output_dtypes=[float],join="override")
        da_der_vslip = xr.apply_ufunc( spline_derivative, da_av_vslip, time_da,
            input_core_dims=[["obs"], ["obs"]], output_core_dims=[["obs"]],
            vectorize=True, dask="parallelized", output_dtypes=[float],join="override")
        
        #calculate slip velocties and force 
        da_uslip = xr.apply_ufunc( spline, da_av_uslip, time_da,
            input_core_dims=[["obs"], ["obs"]], output_core_dims=[["obs"]],
            vectorize=True, dask="parallelized", output_dtypes=[float],join="override")
        da_vslip = xr.apply_ufunc( spline, da_av_vslip, time_da,
            input_core_dims=[["obs"], ["obs"]], output_core_dims=[["obs"]],
            vectorize=True, dask="parallelized", output_dtypes=[float],join="override")
        
        da_Uslip = np.sqrt(da_uslip**2 + da_vslip**2)
        da_rep = Re_particle(da_Uslip,diameter,kinematic_viscosity_water)

        da_Fslip = da_Uslip*factor_drag_white1991(da_rep)
    

        da_der_uslip_nwindow = make_windowed_da_dask(da_der_uslip,Nwindow+1)
        da_der_vslip_nwindow = make_windowed_da_dask(da_der_vslip,Nwindow+1)
        da_Rep_nwindow = make_windowed_da_dask(da_rep, Nwindow+1)
        da_u_f_mei1992 = xr.apply_ufunc(
            f_Mei1992_unit_vectorized_numba_Rep_array,
            da_der_uslip_nwindow,      # dudt: (trajectory, obs, nwindow)
            da_Rep_nwindow,            # Rep_arr: same shape
            time_da,                   # t: (obs,) or (trajectory, obs)
            kwargs=dict(
                c1=cs['Dorgan']['c1'],
                c2=cs['Dorgan']['c2'],
                h=dt,
                tau_diff=tau_diff
            ),
            input_core_dims=[["nwindow"], ["nwindow"], []],
            output_core_dims=[["nwindow"]],
            vectorize=True,
            dask="parallelized",
            output_dtypes=[float],
        )
        da_v_f_mei1992 = xr.apply_ufunc(
            f_Mei1992_unit_vectorized_numba_Rep_array,
            da_der_vslip_nwindow,      # dudt: (trajectory, obs, nwindow)
            da_Rep_nwindow,            # Rep_arr: same shape
            time_da,                   # t: (obs,) or (trajectory, obs)
            kwargs=dict(
                c1=cs['Dorgan']['c1'],
                c2=cs['Dorgan']['c2'],
                h=dt,
                tau_diff=tau_diff
            ),
            input_core_dims=[["nwindow"], ["nwindow"], []],
            output_core_dims=[["nwindow"]],
            vectorize=True,
            dask="parallelized",
            output_dtypes=[float],
        )

        da_u_hinsberg = xr.apply_ufunc(
            Hinsberg_vectorized_numba,
            da_u_f_mei1992,
            kwargs=dict(N=Nwindow, h=dt),
            input_core_dims=[["nwindow"]],
            output_core_dims=[[]],
            vectorize=True,
            dask="parallelized",
            output_dtypes=[float],
        )
        da_v_hinsberg = xr.apply_ufunc(
            Hinsberg_vectorized_numba,
            da_v_f_mei1992,
            kwargs=dict(N=Nwindow, h=dt),
            input_core_dims=[["nwindow"]],
            output_core_dims=[[]],
            vectorize=True,
            dask="parallelized",
            output_dtypes=[float],
        )
        da_u_history = np.sqrt(tau_diff / (4 * np.pi)) * da_u_hinsberg
        ds_v_history = np.sqrt(tau_diff / (4 * np.pi)) * da_v_hinsberg
        da_F_h = np.sqrt(da_u_history**2 + ds_v_history**2)
        da_ratio = da_F_h/da_Fslip
        array = da_ratio.values.flatten()
        array = array[~np.isnan(array)]
        data_history_slip[pt][None]=array
        bins,pdf = make_lognormal_PDF(array,nbins=100,norm=True,vmin=1e-4,vmax=1e2)
        ax.plot(bins,pdf,'--',color='cornflowerblue',zorder=20)

ax.set_ylabel('PDF')
ax.set_xlabel('history force / drag force')
ax.set_xscale('log')
ax.legend(['flexible Re$_p$','Re$_p$=0','Re$_p$=10','Re$_p$=100','Re$_p$=450','Re$_p$=1000'])

fig.tight_layout()
fig.savefig('../figures/history_force/pdf_history_force_drag_force.pdf')
fig.savefig('../figures/history_force/pdf_history_force_drag_force.png')

# 

In [ ]:
fig, ax = plt.subplots()

for pt in particle_types:
    if(pt == 'inertial_Rep_constant'):
        for Rep, color in zip(Replist,Magmalist):
            print(np.mean(data_history_slip[pt][Rep][0:50000]))
            bins,pdf = make_lognormal_PDF( data_history_slip[pt][Rep],nbins=200,norm=True,vmin=1e-5,vmax=1e1)
            ax.plot(bins,pdf,'-.',color=color, zorder=-Rep)
    else:
        print(np.mean(data_history_slip[pt][None][0:50000]))
        bins,pdf = make_lognormal_PDF( data_history_slip[pt][None],nbins=200,norm=True,vmin=1e-5,vmax=1e1)
        ax.plot(bins,pdf,'--',color='cornflowerblue')


ax.set_ylabel('PDF')
ax.set_xlabel('history term / drag term')
ax.set_xscale('log')
# ax.set_yscale('log')
ax.legend(['flexible Re$_p$','Re$_p$=0','Re$_p$=10','Re$_p$=100','Re$_p$=450','Re$_p$=1000'])

fig.tight_layout()
fig.savefig('../figures/history_force/pdf_history_force_drag_force.pdf')
fig.savefig('../figures/history_force/pdf_history_force_drag_force.png')
        

In [ ]:
np.mean(data_history_slip[pt][Rep][-50000:])

In [ ]:
data_history_slip[pt][Rep][-100:]

In [ ]:
# calculate history term with mei history kernel loop over trajectories

fig,ax=plt.subplots()
# ax2=ax.twinx()

Tmax = 12 * 47
nav = 12#12
nresample = 1
dt =300
dt_resample = dt/nresample
tau_diff = diffusion_time(diameter,kinematic_viscosity_water) # in sec
# loop over all particles

pt = 'inertial_Rep_constant'
Rep =0
print(Rep)
ratio_array = []
# loop over trajectories
id =2
ds = data[pt][coriolis][Rep][date].isel(trajectory=id)
    # calculate window averages
da_av_uslip = calc_tidal_av(ds.uslip,nav)
da_av_vslip = calc_tidal_av(ds.vslip,nav)
# make reshifted time array
time = ds.time[nav+1:Tmax].values-dt *nav /2

# make reshifted, resampled time array
time_resample = np.arange(time[0],time[-1],dt_resample)

# load data to array to put in to cubicspline
av_uslip =  da_av_uslip[nav+1:Tmax].values
av_vslip =  da_av_vslip[nav+1:Tmax].values

# calculate cubicspline for data
spl_uslip = CubicSpline(time, av_uslip)
spl_vslip = CubicSpline(time, av_vslip)

# calculate derivatives from the cubicplines
der_uslip = spl_uslip.derivative(1)
der_vslip = spl_vslip.derivative(1)

# sample derivative into array using the resampled time
array_duslipdt = der_uslip(time_resample)
array_dvslipdt = der_vslip(time_resample)


# # calculate timescale, for rep = 0 officially infinite so ure rep = 1 instead
# if(Rep == 0 ):
#     tau_H= history_timescale(1,tau_diff)
# else:
#     tau_H = 1* history_timescale(Rep,tau_diff) # in sec
tau_H = 1*3600 # or use constant time for all values? -

# define number of timesteps when calculating history term
Nwindow = int(tau_H/dt_resample)

# define maximum index number for calculating history term
Nmax = int((Tmax-nav-1)*nresample)


# define timelist given the maximum index number
Tlist = np.arange(0,Nmax,1)*dt_resample

#create empty list to save history force into
F_H_Nwindow = []

# create emptly list to save time values into


# loop over original timesteps to calculate history term
for N in range (Nwindow+nresample,Nmax-nresample,nresample):

    # calculate x and y component history force
    F_H_x =History_Force_Hinsberg_Mei_kernel(array_duslipdt[N-Nwindow-1:N+1],Tlist[N],cs['Dorgan']['c1'],cs['Dorgan']['c2'],Rep,Nwindow,dt_resample,diameter,kinematic_viscosity_water)
    F_H_y =History_Force_Hinsberg_Mei_kernel(array_dvslipdt[N-Nwindow-1:N+1],Tlist[N],cs['Dorgan']['c1'],cs['Dorgan']['c2'],Rep,Nwindow,dt_resample,diameter,kinematic_viscosity_water)
    # calculate magnitude of force
    F_H = np.sqrt(F_H_x**2+F_H_y**2)

    # write magnitude to list
    F_H_Nwindow.append(F_H_x)
    # write timestep to lists


# make np array from lists
F_H_Nwindow = np.asarray(F_H_Nwindow)
ax.plot(Tlist[Nwindow+nresample:Nmax-nresample:nresample],F_H_Nwindow,color='navy')


ax.plot(time_da, da_history[2],color='firebrick')


In [ ]:
       # da_uslip = xr.apply_ufunc( spline, da_av_uslip, time_da,
            #     input_core_dims=[["obs"], ["obs"]], output_core_dims=[["obs"]],
            #     vectorize=True, dask="parallelized", output_dtypes=[float],join="override")
            # da_vslip = xr.apply_ufunc( spline, da_av_vslip, time_da,
            #     input_core_dims=[["obs"], ["obs"]], output_core_dims=[["obs"]],
            #     vectorize=True, dask="parallelized", output_dtypes=[float],join="override")
            
            # da_Uslip = da_uslip**2 + da_vslip**2
            # da_rep = Re_particle(da_Uslip,diameter,kinematic_viscosity_water)
            # Tlist_xr = xr.DataArray(Tlist, dims=['obs'])

In [ ]:
test = spline_derivative(da_av_vslip[2].values,time_da.values)
print(test)

In [ ]:
def make_windowed_da(da: xr.DataArray, nwindow: int) -> xr.DataArray:
    traj_dim = 'trajectory'
    obs_dim = 'obs'

    n_traj, n_obs = da.shape

    # Pad with zeros on the left (before obs=0)
    padded = np.pad(da.values, ((0, 0), (nwindow - 1, 0)), mode='constant')

    # Create windowed array
    result = np.lib.stride_tricks.sliding_window_view(padded, window_shape=nwindow, axis=1)

    # result shape: (trajectory, obs, nwindow)
    coords = {
        'trajectory': da.coords[traj_dim],
        'obs': da.coords[obs_dim],
        'nwindow': np.arange(nwindow)
    }

    return xr.DataArray(result, dims=(traj_dim, obs_dim, 'nwindow'), coords=coords)


da = xr.DataArray(
    np.arange(2*5).reshape(2,5),
    dims=['trajectory', 'obs'],
    coords={'trajectory': [0,1], 'obs': [0,1,2,3,4]}
)

result=make_windowed_da(da,2)

In [ ]:
print(da)
print(result)

In [ ]:
# Mock input
n_trajectories, n_obs = 1000, 100
times = np.linspace(0, 10, n_obs)  # time array (1D)

# Simulated velocities: shape (trajectory, obs)
velocities = xr.DataArray(
    np.random.rand(n_trajectories, n_obs),
    dims=("trajectory", "obs"),
    coords={"trajectory": np.arange(n_trajectories), "obs": times},
    name="velocity"
)

# Define a function that works on 1D velocity arrays
def spline_derivative(v, t):
    cs = CubicSpline(t, v)
    return cs.derivative()(t)

# Apply the function over each trajectory using apply_ufunc
velocity_deriv = xr.apply_ufunc(
    spline_derivative,
    velocities,
    times,
    input_core_dims=[["obs"], ["obs"]],
    output_core_dims=[["obs"]],
    vectorize=True,
    dask="parallelized",  # Optional: enable if using Dask
    output_dtypes=[float]
)

velocity_deriv.name = "velocity_derivative"

In [ ]:
rep_measured.size

In [ ]:
np.size([rep_measured])

In [ ]:
# I have to check what I am actually doing here, what is the window value you need to use? I think something goes right as basset and mei kernel give same value for Rep = 0
# check diffent values of windows? actually take some time for this

fig,ax=plt.subplots()
ax2=ax.twinx()
id = 2
Tmax = 12 * 47
nav = 1# 12
pt = 'inertial_Rep_constant'
nresample = 100
dt =300
tau_diff = diffusion_time(diameter,kinematic_viscosity_water) # in sec
for Rep, color in zip(Replist,Magmalist):
    if(Rep ==0):
        continue
    # print(Rep)
    ds = data[pt][coriolis][Rep][date].isel(trajectory=id)
    da_av_uslip = calc_tidal_av(ds.uslip,nav)
    da_av_vslip = calc_tidal_av(ds.vslip,nav)
    time = ds.time[nav+1:Tmax].values-dt *nav /2
    
    # print(dt)
    dt_resample = dt/nresample
        # print(dt_resample)
    time_resample = np.arange(time[0],time[-1],dt_resample)

    av_uslip =  da_av_uslip[nav+1:Tmax].values
    av_vslip =  da_av_vslip[nav+1:Tmax].values
    spl_uslip = CubicSpline(time, av_uslip)
    spl_vslip = CubicSpline(time, av_vslip)
    der_uslip = spl_uslip.derivative(1)
    der_vslip = spl_vslip.derivative(1)
    array_duslipdt = der_uslip(time_resample)
    array_dvslipdt = der_vslip(time_resample)

    tau_H = 3 * history_timescale(Rep,tau_diff) # in sec
    # print(tau_H)
    Nwindow = int(tau_H/dt_resample)
    Nmax = int((Tmax-nav-1)*nresample)
    # print(array_duslipdt.size)
    # print(Nmax)

    Tlist = np.arange(0,Nmax,1)*dt_resample

    F_B_Nwindow = []
    tvalues = []
  
    print(Nwindow)
    for N in range (Nwindow+nresample,Nmax-nresample,nresample):
        T=Tlist[N]
        # print(array_duslipdt[N-Nwindow-1:N+1].size)
        F_H =History_Force_Hinsberg_Mei_kernel(array_dvslipdt[N-Nwindow-1:N+1],T,cs['Dorgan'][c1],cs['Dorgan'][c2],Nwindow,dt_resample,diameter,kinematic_viscosity_water)
        F_B = History_Force_Hinsberg_Basset_kernel(array_dvslipdt[N-Nwindow-1:N+1],T,Nwindow,dt_resample,diameter,kinematic_viscosity_water)
       
        F_B_Nwindow.append(F_B)
    
        tvalues.append(T)

    F_B_Nwindow = np.asarray(F_B_Nwindow)

    tvalues = np.asarray(tvalues)
    ax.plot(tvalues/3600,F_B_Nwindow,'-',color=color,alpha=1)

    ax.plot(tvalues/3600,spl_vslip(tvalues)*factor_drag_white1991(Rep),'--',color=color)
    # ax.plot(tvalues/3600,15*der_uslip(tvalues),':',color=color)

    #     print(f'y.size = {y.size}')
#     # print(y)
#     tau_H = history_timescale(Rep,tau_diff) # in sec
#     # print(tau_H)
#     Nwindow =int(tau_H/dt_resample)
#     # print(Nwindow)
#     # F_H_Nwindow = []
#     # F_H_Nwindow_Dorgan = []
#     F_B_Nwindow = []

#     Tlist = np.arange(1*tau_H+dt, time[-1],dt)
#     # N = 
#     # print(Tlist/3600)
#     i=0
#     for T in Tlist:
#         N = int((T-1*tau_H+dt)/dt_resample)
#         print(f'N-Nwindow-1 = {N-Nwindow-1}')
#         print(f'N+1 ={N+1}')

#         # print(i)
#         F_B = History_Force_Hinsberg_Basset_kernel(y[N-Nwindow-1:N+1],T,Nwindow,dt_resample,diameter,kinematic_viscosity_water,rho_water)
        
#         F_B_Nwindow.append(F_B)
#         i+=1

#     # F_H_Nwindow = np.asarray(F_H_Nwindow)
#     F_B_Nwindow = np.asarray(F_B_Nwindow)

#     ax.plot(Tlist,F_B_Nwindow,'-',color=color,alpha=1)
#     # ax.plot(Nwindowlist,F_B_Nwindow,'--',color=color)
# # print(dt)
# # print(T)



In [ ]:
# calculate histoy force using history time as window time 

print(f'tau_diff = {tau_diff:.2f} s') 
for Rep in Replist:
    if(Rep==0):
        continue
    tau_H = history_timescale(Rep,tau_diff) # in sec
    
    History_Force_Hinsberg_Basset_kernel((y[N-Nwindow-1:N+1],T,Nwindow,dt,diameter,kinematic_viscosity_water,rho_water))
    

In [ ]:
12*46

In [ ]:
# plot values
id = 2
dt_write =10
fig,ax=plt.subplots()
markers=['-','--','-','--','-','--','-','--','-','--','-','--']
for pt, marker in zip(particle_types,markers):
    # print(pt)
    if(pt in ('inertial_Rep_constant')):
        print(pt)
        for Rep, color in zip(Replist,Magmalist):
            print(Rep)
            ax.plot(data[pt][coriolis][Rep][date].time[id,24:]/3600,data[pt][coriolis][Rep][date].duslipdt[id,24:],'--o',color=color)
            
    # else:
        # ax.plot(data[pt][coriolis][None][date].time[id,10:]/3600,data[pt][coriolis][None][date].duslipdt[id,10:],marker,color='cornflowerblue')



# ax.set_yscale('log')
ax.set_xlabel('time [hours]')
ax.set_ylabel('$du_{\\mathrm{slip}}/dt$ [m/s$^2$]')
# ax.set_ylim(-1.1e-6,2.2e-6)

In [ ]:
# plot values
id = 2
dt_write =10
fig,ax=plt.subplots()
for pt in ['inertial_Rep_constant']:
    
    ax.plot(data_short[pt][dt_write].time[id,10:]/60,data_short[pt][dt_write].uparticle[id,10:],'.',color='purple')
    ax.plot(data_short[pt][dt_write].time[id,10:]/60,data_short[pt][dt_write].ufluid[id,10:],'.',color='c')
    
    ax2 = ax.twinx()
    ax2.plot(data_short[pt][dt_write].time[id,10:]/60,data_short[pt][dt_write].uslip[id,10:],'.',color='orange')

ax.legend(['$u_{\\mathrm{particle}}$','$u_{\\mathrm{fluid}}$'],loc=(1.2,0.3))

ax2.legend(['$u_{\\mathrm{slip}}$'],loc=(1.2,0.22))

ax.set_ylabel('velocity particle or fluid')
ax2.set_ylabel('slip velocity')
ax.set_xlabel('time [minutes]')
ax.axvline(60,zorder=-5,color='grey')
# ax.set_yscale('log')


In [ ]:
# calculating history term for 
id = 500
pt = 'inertial_drag_Rep'
ds = data_short[pt][dt_write].isel(trajectory=id)
Tend=(ds.time.ffill(dim='obs')[-1]-ds.time[0]).values
T = 6000
dt = dt = (ds.time[1]-ds.time[0]).values
N=int(T/dt)
time = np.arange(0,T+0.1*dt,dt)
y = ds.duslipdt[0:N+1].values
print(ds.uf[N].values)
print(y[N])
spl = CubicSpline(time, y)
time_new =np.arange(0,T,1)
y_new =spl(time_new)
Nnew=N*10


Nwindowlist = np.arange(10,N-10,1)
F_H_Nwindow = []
F_H_Nwindow_Dorgan = []
F_B_Nwindow = []
for Nwindow in Nwindowlist:


    # F_H = History_Force_Hinsberg_Mei_kernel(y[N-Nwindow-1:N+1],T,cs['Mei']['c1'],cs['Mei']['c2'],Rep,Nwindow,dt,diameter,kinematic_viscosity_water,rho_water)
    F_H = History_Force_Hinsberg_Mei_kernel(y[N-Nwindow-1:N+1],T,cs['Dorgan']['c1'],cs['Dorgan']['c2'],Rep,Nwindow,dt,diameter,kinematic_viscosity_water,rho_water)
    # F_B = History_Force_Hinsberg_Basset_kernel(y[N-Nwindow-1:N+1],T,Nwindow,dt,diameter,kinematic_viscosity_water,rho_water)
    F_H_Nwindow.append(F_H)
    # F_H_Nwindow_Dorgan.append(F_H_Dorgan)
    # F_B_Nwindow.append(F_B)

F_H_Nwindow = np.asarray(F_H_Nwindow)
# F_B_Nwindow = np.asarray(F_B_Nwindow)
# F_H_Nwindow_Dorgan = np.asarray(F_H_Nwindow_Dorgan)
Fdrag = slip_force(ds.uf,diameter,kinematic_viscosity_water,rho_water,Rep).values
print()
fig,ax=plt.subplots()
# ax.plot(Nwindowlist*dt,F_H_Nwindow,'o')
ax.plot(Nwindowlist*dt,F_H_Nwindow/Fdrag[N+1],'o')
# ax.plot(Nwindowlist*dt,F_B_Nwindow/Fdrag[N+1],'o')




In [ ]:
# check criterium 
# tau_h * |duslip/dt| / |uslip| << 1 then history term negible
def history_number (duslipdt, uslip, tau_H):
    return tau_H * duslipdt/uslip
id = 0
dt_write=10

duslipdtmean = data_short['inertial_drag_Rep'][dt_write].duslipdt.pipe(abs).mean(dim='obs').values
uslipmean = data_short['inertial_drag_Rep'][dt_write].uf.pipe(abs).mean(dim='obs').values

history_number_per_particle = history_number(duslipdtmean,uslipmean,tau_H)
history_number_per_particle = history_number_per_particle[~np.isnan(history_number_per_particle)]
print(np.mean(history_number_per_particle))
bins,pdf = make_PDF(history_number_per_particle,nbins=200,norm=True)

fig,ax=plt.subplots()
# ax.plot(history_number(duslipdtmean,uslipmean,tau_H),'o')
# ax.set_yscale('log')
ax.set_xscale('log')
ax.plot(bins,pdf,'-o')
# history_number()

In [ ]:
# calculating history term for 
id = 0
dt_write=10
pt = 'inertial_drag_Rep'
ds = data_short[pt][dt_write].isel(trajectory=id)
Tend=(ds.time.ffill(dim='obs')[-1]-ds.time[0]).values
T = 6000
dt = dt = (ds.time[1]-ds.time[0]).values
N=int(T/dt)
time = np.arange(0,T+0.1*dt,dt)
x = ds.duslipdt[0:N+1].values
y = ds.dvslipdt[0:N+1].values

Nwindow =  int(tau_H/dt)

Nwindowlist = np.arange(10,N-10,1)
F_H_v_Nwindow = []
F_H_u_Nwindow = []

for n in range(100+Nwindow,N,1):
    F_H_u = History_Force_Hinsberg_Mei_kernel(x[n-Nwindow-1:n+1],time[n+1],cs['Dorgan']['c1'],cs['Dorgan']['c2'],Rep,Nwindow,dt,diameter,kinematic_viscosity_water,rho_water)
    F_H_u_Nwindow.append(F_H_u)
    F_H_v = History_Force_Hinsberg_Mei_kernel(y[n-Nwindow-1:n+1],time[n+1],cs['Dorgan']['c1'],cs['Dorgan']['c2'],Rep,Nwindow,dt,diameter,kinematic_viscosity_water,rho_water)
    F_H_v_Nwindow.append(F_H_v)


F_H_u_Nwindow = np.asarray(F_H_u_Nwindow)
F_H_v_Nwindow = np.asarray(F_H_v_Nwindow)


Fdrag_u = slip_force(ds.uf,diameter,kinematic_viscosity_water,rho_water,Rep).values
Fdrag_v = slip_force(ds.vf,diameter,kinematic_viscosity_water,rho_water,Rep).values
norm =np.mean(np.sqrt(Fdrag_u[100+Nwindow+1:N+1]**2 + Fdrag_v[100+Nwindow+1:N+1]**2))

fig,ax=plt.subplots()

ax.plot(time[100+Nwindow+1:N+1]/60,Fdrag_u[100+Nwindow+1:N+1]/norm,'-',color='firebrick') 
ax.plot(time[100+Nwindow+1:N+1]/60,Fdrag_v[100+Nwindow+1:N+1]/norm,'--',color='firebrick') 

ax.plot(time[100+Nwindow+1:N+1]/60,10*ds.fcor_u[100+Nwindow+1:N+1]/norm,'-',color='green') 
ax.plot(time[100+Nwindow+1:N+1]/60,10*ds.fcor_v[100+Nwindow+1:N+1]/norm,'--',color='green') 

ax.plot(time[100+Nwindow+1:N+1]/60,100*F_H_u_Nwindow/norm,'-',color='navy') #Fdrag_u[100+Nwindow+1:N+1]
ax.plot(time[100+Nwindow+1:N+1]/60,100*F_H_v_Nwindow/norm,'--',color='navy') #/Fdrag_v[100+Nwindow+1:N+1]

ax.legend(['$F^{\\phi}_{drag}$','$F^{\\lambda}_{drag}$', ' $10 \\times F^{\\phi}_{\\mathrm{cor}}$', ' $10 \\times F^{\\lambda}_{\\mathrm{cor}}$',  '$100 \\times F^{\\phi}_{\\mathrm{his}}$','$100 \\times F^{\\lambda}_{\\mathrm{his}}$' ], loc = (1,0.2))
ax.set_xlabel('time [minutes]')
ax.set_ylabel('Force [$\\langle|\\boldsymbol{F}_{\\mathrm{drag}}|\\rangle$]')
# a bit suspisous that the lines are this straigt, but at least it is nice to show that 
# drag force = 10 x coriolis = 100 x history force
# does not even have to be completely correct but it would be an order of magnitude estimate 
# -> maybe also calculate effect on direction

# ax2=ax.twinx()
# up = ds.lon[101:N-5].values-ds.lon[100:N-6].values
# ax2.plot(time[101:N-5]/60,up,'o',color='orange')
# ax2.plot(time[100+Nwindow+1:N+1]/60,10*ds.lat[100+Nwindow+1:N+1],color='orange')


ax.axvline(60,color='grey',zorder=-10)

# 
